In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import numpy as np
from Module.sam import sam_model_registry, SamPredictor
from Module.sam.utils.transforms import ResizeLongestSide

C:\Users\danusys16\anaconda3\envs\yolov8\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\danusys16\anaconda3\envs\yolov8\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
D:\015_segmentation\Custom_SAM\Module\sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with Module.sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
D:\015_segmentation\Custom_SAM\Module\sam\modeling\tiny_vit_sam.py:656: UserWarning: Over

## Model

In [16]:
model_type = 'vit_t'
checkpoint = "../runs/241119_vit-b_to_vit-t_DANU_WS_v2/tiny_vit_best.pth"

In [17]:
save_dir = os.path.split(checkpoint)[0]

output_name = os.path.split(checkpoint)[1]
output_name = os.path.splitext(output_name)[0]+"_encoder.onnx" 

output_path = os.path.join(save_dir, output_name)

In [18]:
sam = sam_model_registry[model_type](checkpoint=checkpoint)
transform = ResizeLongestSide(sam.image_encoder.img_size)

pixel_mean = torch.Tensor([123.675, 116.28, 103.53]).view(-1, 1, 1)
pixel_std = torch.Tensor([58.395, 57.12, 57.375]).view(-1, 1, 1)

In [19]:
dummy_input = np.zeros((sam.image_encoder.img_size, sam.image_encoder.img_size, 3), dtype=np.uint8)
dummy_input = transform.apply_image(dummy_input)
dummy_input = torch.as_tensor(dummy_input, device='cpu')
dummy_input = dummy_input.permute(2, 0, 1).contiguous()[None, :, :, :]

#dummy_input = (dummy_input - pixel_mean) / pixel_std

In [20]:
dummy_input.shape

torch.Size([1, 3, 1024, 1024])

In [21]:
output_names = ['image_embedding']

In [22]:
class Model(torch.nn.Module):
    def __init__(self, image_size, checkpoint, model_type):
        super().__init__()
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint)
        self.sam.to(device='cpu')
        self.predictor = SamPredictor(self.sam)
        self.image_size = image_size

    def forward(self, x):
        self.predictor.set_torch_image(x, (self.image_size))
        return self.predictor.get_image_embedding()


In [23]:
sam_model =Model(image_size=(1024,1024),
                checkpoint=checkpoint,
                model_type=model_type)

In [24]:
model_trace = torch.jit.trace(sam_model, dummy_input)

D:\015_segmentation\Custom_SAM\Module\sam\predictor.py:81: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  and transformed_image.shape[1] == 3
D:\015_segmentation\Custom_SAM\Module\sam\predictor.py:82: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  and max(*transformed_image.shape[2:]) == self.model.image_encoder.img_size
D:\015_segmentation\Custom_SAM\Module\sam\modeling\tiny_vit_sam.py:338: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be t

In [25]:
torch.onnx.export(model_trace, 
                  dummy_input, 
                  output_path,
                  input_names=['input'], 
                  output_names=output_names)

C:\Users\danusys16\anaconda3\envs\yolov8\lib\site-packages\torch\onnx\utils.py:847: UserWarning: no signature found for <torch.ScriptMethod object at 0x0000025ECDC419F0>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")
C:\Users\danusys16\anaconda3\envs\yolov8\lib\site-packages\torch\onnx\_internal\jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
C:\Users\danusys16\anaconda3\envs\yolov8\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\p

In [26]:
output_path

'../runs/241119_vit-b_to_vit-t_DANU_WS_v2\\tiny_vit_best_encoder.onnx'